In [7]:
#!/usr/bin/env sage
# Standalone Ciminion implementation and polynomial system extractor
# No external file dependencies

class Ciminion:
    """
    Ciminion stream cipher for polynomial system extraction
    
    Ciminion processes plaintext in 2-element blocks:
    - First block: [nonce, k0, k1] -> pC -> pE -> encrypt with plaintext[0:2]
    - Later blocks: update state with new keys, apply rol(), then pE -> encrypt
    
    Supports arbitrary number of blocks (plaintext must have even length)
    """
    
    def __init__(self, field, constants, master_key, N=None, R=None, IV=1, round_keys=None):
        """
        Initialize Ciminion cipher
        
        Args:
            field: Finite field or polynomial ring
            constants: Round constants (4 per round)
            master_key: 2-element master key
            N: Number of rounds for pC (computed if None)
            R: Number of rounds for pE (computed if None)
            IV: Initialization vector
            round_keys: Pre-supplied round keys (optional)
        """
        if field.is_field():
            assert field.is_prime_field(), f"Ciminion only works over prime fields, not {field}"
        else:
            assert field.is_ring(), f"Field must be field or polynomial ring, not {type(field)}"
            assert field.base_ring().is_prime_field(), f"Base ring must be prime field"
        
        # Simplified security parameters
        if N is None:
            N = 3  # Small default for demo
        if R is None:
            R = 3  # Small default for demo
        
        assert len(constants) == 4 * (N + R), f"Need {4*(N+R)} constants, got {len(constants)}"
        assert len(master_key) == 2, f"Master key must have 2 elements, got {len(master_key)}"
        assert N > 0 and R > 0, f"N={N}, R={R} must be positive"
        assert all(constants[4*i + 3] not in [0, 1] for i in range(N + R)), "RC4i constants must be ≠ 0,1"
        
        if round_keys:
            round_keys = [field(k) for k in round_keys]
        
        self.field = field
        self.constants = [field(c) for c in constants]
        self._master_key = [field(mk) for mk in master_key]
        self.N = N
        self.R = R
        self.IV = field(IV)
        self._round_keys = round_keys

    def round_function(self, round_idx, state):
        """
        Apply ith round function f_i to state
        
        Args:
            round_idx: Round index
            state: 3-element state vector or list
            
        Returns:
            New state after round function
        """
        assert round_idx < self.N + self.R, f"Round {round_idx} not defined"
        assert len(state) == 3, f"State must have 3 elements, got {len(state)}"
        
        field = self.field
        constants = self.constants
        a, b, c = [field(s) for s in state]
        
        # Round function: new_state = [a, b, a*b + c]
        new_state = [a, b, a*b + c]
        
        # Apply linear transformation with round constants
        rc4 = constants[4 * round_idx + 3]
        # mult_matrix = [[0, 0, 1], [1, rc4, rc4], [0, 1, 1]]
        # round_constants = [constants[4*round_idx + 2], constants[4*round_idx + 0], constants[4*round_idx + 1]]
        
        transformed_state = [
            new_state[2] + constants[4 * round_idx + 2],                    # 0*a + 0*b + 1*c + rc2
            new_state[0] + rc4 * new_state[1] + rc4 * new_state[2] + constants[4 * round_idx + 0],  # 1*a + rc4*b + rc4*c + rc0  
            new_state[1] + new_state[2] + constants[4 * round_idx + 1]      # 0*a + 1*b + 1*c + rc1
        ]
        
        return transformed_state

    def apply_rounds(self, state, starting_round, num_rounds):
        """Apply multiple round functions"""
        assert len(state) == 3, f"State must have 3 elements"
        
        field = self.field
        new_state = [field(s) for s in state]
        
        for i in range(num_rounds):
            new_state = self.round_function(starting_round + i, new_state)
        
        return new_state

    def pc(self, state):
        """Apply pC permutation (first N rounds)"""
        return self.apply_rounds(state, 0, self.N)

    def pe(self, state):
        """Apply pE permutation (last R rounds)"""
        return self.apply_rounds(state, self.N, self.R)

    def rol(self, state):
        """Rolling function (Toffoli gate): [a, b, c] -> [a*b + c, a, b]"""
        assert len(state) == 3, f"State must have 3 elements"
        
        field = self.field
        a, b, c = [field(s) for s in state]
        return [a*b + c, a, b]

    def encrypt(self, nonce, plaintext, round_keys_symbolic=None):
        """
        Encrypt arbitrary-length plaintext (must be even length)
        
        Args:
            nonce: Nonce value
            plaintext: List of plaintext elements (even length)
            round_keys_symbolic: Optional list of symbolic round keys
            
        Returns:
            Ciphertext list
        """
        assert len(plaintext) % 2 == 0, f"Plaintext length must be even, got {len(plaintext)}"
        assert len(plaintext) > 0, "Plaintext cannot be empty"
        
        field = self.field
        nonce = field(nonce)
        plaintext = [field(pt) for pt in plaintext]
        num_blocks = len(plaintext) // 2
        
        # Initialize key schedule or use symbolic keys
        if round_keys_symbolic:
            assert len(round_keys_symbolic) >= 2 + 2*(num_blocks-1), f"Need at least {2 + 2*(num_blocks-1)} keys for {num_blocks} blocks"
            key_idx = 0
            k_0 = round_keys_symbolic[key_idx]; key_idx += 1
            k_1 = round_keys_symbolic[key_idx]; key_idx += 1
        else:
            # Use master key schedule (dummy implementation)
            k_0 = self._master_key[0]
            k_1 = self._master_key[1]
        
        # Initial state: [nonce, k_0, k_1]
        initial_state = [nonce, k_0, k_1]
        
        # Apply pC then pE for first block
        middle_state = self.pc(initial_state)
        out_state = self.pe(middle_state)
        
        # First ciphertext block
        ciphertext = [out_state[0] + plaintext[0], out_state[1] + plaintext[1]]
        
        # Process remaining blocks
        for block_idx in range(1, num_blocks):
            # Get next round keys
            if round_keys_symbolic:
                k_new1 = round_keys_symbolic[key_idx]; key_idx += 1
                k_new2 = round_keys_symbolic[key_idx]; key_idx += 1
            else:
                # Simple key schedule for demo
                k_new1 = self._master_key[0] * (block_idx + 1)
                k_new2 = self._master_key[1] * (block_idx + 1)
            
            # Update middle state with new round keys
            middle_state[1] += k_new1
            middle_state[2] += k_new2
            
            # Apply rolling function
            middle_state = self.rol(middle_state)
            
            # Apply pE
            out_state = self.pe(middle_state)
            
            # Generate next ciphertext block
            pt_idx = block_idx * 2
            ciphertext += [out_state[0] + plaintext[pt_idx], out_state[1] + plaintext[pt_idx + 1]]
        
        return ciphertext


def extract_ciminion_system(prime, N, R, nonce, plaintext, verbose=True):
    """
    Extract polynomial system for Ciminion key recovery from multiple blocks
    """
    if verbose:
        print(f"=== Ciminion Polynomial System Extraction ===")
        print(f"Prime field: F_{prime}")
        print(f"pC rounds (N): {N}, pE rounds (R): {R}")
        print(f"Nonce: {nonce}")
        print(f"Plaintext: {plaintext}")
    
    assert len(plaintext) % 2 == 0, f"Plaintext length must be even"
    num_blocks = len(plaintext) // 2
    
    # Generate constants
    total_rounds = N + R
    constants = []
    for i in range(total_rounds):
        # Generate 4 constants per round, ensuring RC4 ≠ 0,1
        base = (i * 17 + 13) % prime  # Some deterministic generation
        constants.extend([
            (base + 1) % prime,       # RC0
            (base + 7) % prime,       # RC1  
            (base + 23) % prime,      # RC2
            (base + 41) % prime + 2   # RC4 (ensure ≠ 0,1)
        ])
    
    if verbose:
        print(f"Number of blocks: {num_blocks}")
        print(f"Generated {len(constants)} constants")
    
    # Create polynomial ring for symbolic keys
    # We need 2 + 2*(num_blocks-1) keys: k0, k1 for first block, then 2 new keys per additional block
    num_keys = 2 + 2 * (num_blocks - 1)
    ring = PolynomialRing(GF(prime), 'k', num_keys)
    round_keys = ring.gens()
    
    if verbose:
        print(f"Key variables: {list(round_keys)}")
    
    # Create Ciminion instance with polynomial ring
    ciminion = Ciminion(ring, constants, [42, 17], N=N, R=R, IV=1)
    
    # Compute symbolic ciphertext
    symbolic_ciphertext = ciminion.encrypt(nonce, plaintext, round_keys_symbolic=round_keys)
    
    if verbose:
        print(f"\n=== Symbolic Output (Ciphertext as polynomials in keys) ===")
        for i, poly in enumerate(symbolic_ciphertext):
            print(f"c_{i} = {poly}")
        
        print(f"\n=== System Statistics ===")
        degrees = [poly.degree() for poly in symbolic_ciphertext]
        print(f"Degrees: {degrees}")
        print(f"Maximum degree: {max(degrees)}")
        print(f"Total monomials: {sum(len(poly.monomials()) for poly in symbolic_ciphertext)}")
    
    return symbolic_ciphertext, ring, constants


def create_key_recovery_system(prime, N, R, nonce, plaintext, target_ciphertext, verbose=True):
    """
    Create key recovery system: given plaintext/ciphertext pair, find keys
    """
    if verbose:
        print(f"\n=== Creating Key Recovery System ===")
    
    # Get symbolic system
    symbolic_ciphertext, ring, constants = extract_ciminion_system(prime, N, R, nonce, plaintext, verbose=False)
    
    if verbose:
        print(f"Target ciphertext: {target_ciphertext}")
    
    # Create attack equations: symbolic_ciphertext[i] - target_ciphertext[i] = 0
    attack_equations = []
    for i, (symbolic, target) in enumerate(zip(symbolic_ciphertext, target_ciphertext)):
        equation = symbolic - target
        attack_equations.append(equation)
        if verbose:
            print(f"Equation {i}: {equation} = 0")
    
    if verbose:
        print(f"\nKey recovery system: {len(attack_equations)} equations in {ring.ngens()} unknowns")
        degrees = [eq.degree() for eq in attack_equations]
        print(f"Equation degrees: {degrees}")
        print(f"Maximum degree: {max(degrees)}")
        print(f"Total terms: {sum(len(eq.monomials()) for eq in attack_equations)}")
    
    return attack_equations, ring, constants


def solve_key_recovery_system(attack_equations, ring, verbose=True):
    """
    Attempt to solve the key recovery system
    """
    if verbose:
        print(f"\n=== Solving Key Recovery System ===")
        print(f"Computing Gröbner basis...")
    
    try:
        import time
        start_time = time.time()
        
        # Compute Gröbner basis
        ideal = Ideal(attack_equations)
        gb = ideal.groebner_basis()
        
        end_time = time.time()
        
        if verbose:
            print(f"Gröbner basis computed in {end_time - start_time:.3f} seconds")
            print(f"Gröbner basis has {len(gb)} polynomials")
        
        if gb:
            gb_degrees = [p.degree() for p in gb if p != 0]
            if gb_degrees and verbose:
                print(f"Gröbner basis degrees: {gb_degrees}")
                print(f"Maximum degree in GB: {max(gb_degrees)}")
            
            # Check if system is inconsistent
            if len(gb) == 1 and gb[0] == 1:
                if verbose:
                    print("System is inconsistent (no solutions)")
                return None, gb
            
            # Try to find solutions for small systems
            if len(attack_equations) <= 3:
                try:
                    variety = ideal.variety()
                    if verbose:
                        print(f"Found {len(variety)} solution(s):")
                        for i, sol in enumerate(variety):
                            print(f"  Solution {i}: {sol}")
                    return variety, gb
                except Exception as e:
                    if verbose:
                        print(f"Variety computation failed: {e}")
            else:
                if verbose:
                    print("System too large for variety computation")
            
            return None, gb
        
        else:
            if verbose:
                print("Empty Gröbner basis")
            return None, gb
            
    except Exception as e:
        if verbose:
            print(f"Gröbner basis computation failed: {e}")
        return None, None


def main():
    """
    Main function demonstrating Ciminion polynomial system extraction with multiple blocks
    """
    print("Ciminion Polynomial System Extractor")
    print("=" * 50)
    
    # Small parameters for demonstration
    prime = 101
    
    # Test configurations with different numbers of blocks
    configs = [
        {"N": 1, "R": 1, "blocks": 1, "name": "Minimal (1 block)"},
        {"N": 1, "R": 1, "blocks": 2, "name": "Minimal (2 blocks)"},
        {"N": 2, "R": 1, "blocks": 1, "name": "Small N (1 block)"},
        {"N": 1, "R": 1, "blocks": 3, "name": "Minimal (3 blocks)"},
        {"N": 2, "R": 2, "blocks": 2, "name": "Medium (2 blocks)"},
    ]
    
    # Test data
    nonce = 42
    
    for config in configs:
        print(f"\n{'='*15} {config['name']} Configuration {'='*15}")
        
        N, R, num_blocks = config["N"], config["R"], config["blocks"]
        
        # Create plaintext for the specified number of blocks
        plaintext = list(range(2 * num_blocks))  # Each block has 2 elements
        
        # Create target ciphertext using concrete keys
        F = GF(prime)
        total_rounds = N + R
        constants = []
        for i in range(total_rounds):
            base = (i * 17 + 13) % prime
            constants.extend([
                (base + 1) % prime, (base + 7) % prime, 
                (base + 23) % prime, (base + 41) % prime + 2
            ])
        
        ciminion_concrete = Ciminion(F, constants, [10, 20], N=N, R=R, IV=1)
        target_ciphertext = ciminion_concrete.encrypt(nonce, plaintext)
        
        print(f"Plaintext ({num_blocks} blocks): {plaintext}")
        print(f"Target ciphertext: {target_ciphertext}")
        
        # Extract polynomial system
        symbolic_ciphertext, ring, _ = extract_ciminion_system(prime, N, R, nonce, plaintext, verbose=True)
        
        # Create key recovery system
        attack_equations, ring, _ = create_key_recovery_system(prime, N, R, nonce, plaintext, target_ciphertext, verbose=True)
        
        # For very small systems, try to solve
        if N == 1 and R == 1 and num_blocks == 1:
            solutions, gb = solve_key_recovery_system(attack_equations, ring, verbose=True)
            
            if solutions:
                print(f"\n=== Key Recovery Successful! ===")
                if solutions:
                    sol = solutions[0]
                    recovered_keys = [sol.get(ring.gen(i), None) for i in range(ring.ngens())]
                    print(f"Recovered keys: {recovered_keys}")
                    expected_keys = [10, 20] + [10 * (i+1) for i in range(1, num_blocks)] + [20 * (i+1) for i in range(1, num_blocks)]
                    expected_keys = expected_keys[:ring.ngens()]
                    print(f"Expected keys pattern: {expected_keys}")
        else:
            print(f"\n=== System extracted successfully ===")
            print(f"System ready for analysis with specialized tools")
            print(f"Complexity grows as O(degree^{ring.ngens()}) with {ring.ngens()} key variables")
    
    print(f"\n=== Summary ===")
    print(f"Successfully demonstrated Ciminion polynomial system extraction")
    print(f"- Supports arbitrary number of plaintext blocks (even total length)")
    print(f"- Each additional block requires 2 more key variables")
    print(f"- System complexity grows exponentially with rounds and blocks")
    print(f"- Rolling function creates polynomial dependencies between blocks")


if __name__ == "__main__":
    main()

Ciminion Polynomial System Extractor

=============== Minimal (1 block) Configuration ===============
Plaintext (1 blocks): [0, 1]
Target ciphertext: [46, 31]
=== Ciminion Polynomial System Extraction ===
Prime field: F_101
pC rounds (N): 1, pE rounds (R): 1
Nonce: 42
Plaintext: [0, 1]
Number of blocks: 1
Generated 8 constants
Key variables: [k0, k1]

=== Symbolic Output (Ciphertext as polynomials in keys) ===
c_0 = 35*k0^2 + 13*k0*k1 - 45*k1^2 + k0 - 48*k1 - 32
c_1 = 30*k0^2 + 40*k0*k1 + 48*k1^2 - 43*k0 - 21*k1 - 29

=== System Statistics ===
Degrees: [2, 2]
Maximum degree: 2
Total monomials: 12

=== Creating Key Recovery System ===
Target ciphertext: [46, 31]
Equation 0: 35*k0^2 + 13*k0*k1 - 45*k1^2 + k0 - 48*k1 + 23 = 0
Equation 1: 30*k0^2 + 40*k0*k1 + 48*k1^2 - 43*k0 - 21*k1 + 41 = 0

Key recovery system: 2 equations in 2 unknowns
Equation degrees: [2, 2]
Maximum degree: 2
Total terms: 12

=== Solving Key Recovery System ===
Computing Gröbner basis...
Gröbner basis computed in 0.00